<a href="https://colab.research.google.com/github/srdg/unarchived_ben_tess/blob/main/src/Bengali_Typeset_With_Tesseract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# E2E Training Pipeline
Installs dependencies, language packs;
Gets training tools and training data 
and runs training to make `.traineddata` model. Logs final result after training is completed.

In [ ]:
!echo "*******************Install dependencies***************************"
!sudo apt-get install tesseract-ocr bc libtesseract-dev 
!echo "********************Verify installation**************************"
!which tesseract
!echo "*******************Install language***************************"
!sudo apt-get install tesseract-ocr-ben
!echo "*********************Verify language*************************"
!ls /usr/share/tesseract-ocr/4.00/tessdata | grep traineddata
!echo "******************Download training data********************"
!mkdir -p ./data/ben-ground-truth
!wget https://github.com/srdg/unarchived_ben_tess/\
releases/download/v0.0.3-alpha/data.zip -q --show-progress 
!unzip -q ./data.zip -d ./data/ben-ground-truth && rm -rf ./data.zip
!echo "******************Download training tools********************"
!wget https://github.com/tesseract-ocr/tesstrain/\
archive/master.zip -q --show-progress && unzip -q ./master.zip 
!rm -rf ./master.zip ./sample_data
!echo "***********************Move training  data***********************"
!mv ./data ./tesstrain-master/
!echo "======================= READY TO TRAIN ========================="
%cd ./tesstrain-master/
!echo "======================= TRAINING MODEL ========================="
nohup make training MODEL_NAME=ben RATIO_TRAIN=0.80 LANG_TYPE=Indic  \
DEBUG_INTERVAL=-1 TESSDATA=/usr/share/tesseract-ocr/4.00/tessdata \
MAX_ITERATIONS=99999999 START_MODEL=ben&>data/ben.training.log
!echo "======================= TRAINING DONE! ========================="
from IPython.display import Audio
Audio("../alarm.mp3",autoplay=True)
!tail -10 ./data/ben.training.log

# Data Cleaning Pipeline
Gets data (needs to be uploaded first) and performs hocr ops on it to split images into line-level, converts into .tif format and zips them for download.

In [ ]:
!wget https://github.com/ocropus/hocr-tools/archive/master.zip \
-q --show-progress && unzip master.zip -d . 
!wget https://raw.githubusercontent.com/srdg/\
unarchived_ben_tess/main/src/line_hocr.sh && chmod +xxx *.sh
!apt install tesseract-ocr-ben
!bash line_hocr.sh
%cd data
!rm -rf *.exp0.txt
!mkdir -p data
!mv *.exp0.tif ./data
!zip data.zip ./data/* -q